## Convolutional Autoencoder arquitecture optimization

Con lo concluído en el anterior notebook, nos ocupamos de buscar la arquitectura de nuestro CAE -número de capas convolucionales y densas, número de filtros, funciones de activación, etc.- que mejor performance tenga ante los datos generados en 1.0; estandarizados con nuestro modelo RobustScaler.

In [1]:
import pickle
import os

from matplotlib import pyplot as plt 
import numpy as np
import tensorflow as tf
from tensorflow.keras import losses

from deep_scattering_models.data.create_data import load_data

from deep_scattering_models.models.convolutional_autoencoder import ConvAutoencoder

from deep_scattering_models.visualization.visualize import plot_polarization_signature, plot_history

Cargamos los datos crudos, para poder cargarlos en un pipline que los estandarice, pase a dB y les de forma para ingresar al CAE, en el contexto de una validación cruzada (cada uno de estos pasos debe realizarse en cada iteración, sin cruzar datos entre los folds de entrenamiento y testeo).

In [2]:
raw_data_filename = "raw/spm_signatures_no_noise"
raw_data = load_data(raw_data_filename)

Vamos a realizar un Grid Search barriendo varias arquitecturas posibles. Para eso definimos una grilla o espacio de parámetros, sobre el cual podemos obtener un modelo para cada un de las combinaciones posibles de parámetros y evaluar su performance con nuestros datos. Para que la evaluación sea más robusta, vamos a usar K-fold Cross Validation.

Comenzamos definiendo nuestra grilla de parámetros. Vamos a dejar fijo el parámetro correspondiente a la dimensión del espacio latente, en número bajo para optimizar el resto de la arquitectura -acá suponemos que la performance del modelo va a mejorar para dimensiones más altas del espacio latente-.

In [ ]:
# Fijamos la dimensión del espacio latente
LATENT_DIMENTION = 3

# Configuracion de las capas convolucionales
cv_layers = [
    [(4, (3, 4), 2), (8, (3, 3), 1)],
    [(16, (3, 4), 2), (32, (3, 3), 1)]
    [(32, (5, 6), 2), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 2), (128, (3, 3), 1)],
    [(16, (3, 4), 2), (32, (3, 3), 2), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (2, 2), 2), (128, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (3, 3), 2), (512, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (3, 3), 1), (256, (3, 3), 1)],
    [(128, (3, 4), 2), (256, (2, 2), 1), (256, (1, 1), 1)], 
    [(32, (3, 4), 2), (64, (3, 3), 1), (64, (3, 3), 1), (64, (3, 3), 1)],
    [(32, (3, 4), 2), (64, (3, 3), 2), (128, (3, 3), 2), (256, (3, 3), 1)]
    ]

max_pooling = [True, False]

In [ ]:
# Configuración de las capas densas
dense_layers = [
    (8,), (32,), (64,), (256,),
    (32, 16), (16, 16), (64, 32),
    (256, 128), (512, 256), (32, 32),
    (1024, 512), (512, 64), (256, 32),
    (64, 32, 16), (128, 64, 32), (256, 128, 64),
    (64, 32, 32), (32, 32, 16), (1024, 512, 128),
    (128, 64, 32, 16), (256, 128, 64, 32)
 ]

drop_out = [True, False]